## ETA（End-to-end Target Attention）


<img src="../data/img/hist_model.webp" />

其中影响和提升最大就是DIN->DIEN->MIMN->SIM, 
- DIN/DIEN/MIMN能处理的最大序列长度不超过1000
- SIM能处理的最大长度为54000


**ETA的主要思路是使用SimHash替换SIM GSU中的类别索引方法**



### SimHash

<img src="../data/img/simhash_code.png" style="zoom:50%"/>

上面是SimHash实现的伪代码，其中sgn表示符号函数。每一个行为向量经过一个hash函数后被转变为一个二进制向量，而计算两个行为向量的相似度，便转变为两个二进制向量的汉明距离。


<img src="../data/img/eta_simhash.png" style="zoom:50%"/>


### ETA 模型结构

<img src="../data/img/eta.webp" />

**ETA模型的核心部分是Top-K Retrieval，使用SimHash算法从长序列中提取Top-K个item。每个item使用SimHash生成二进制向量是离线缓存好的，线上预估时只需要计算二进制向量之间的汉明距离。**

<img src="../data/img/eta_exam.png" style="zoom:50%"/>

- 相同序列长度，最好的版本是所有行为输入到Multi-Head，但是耗时最长（v4）。
- 使用向量内积的检索方式比使用SimHash效果更好，但是时间开销也更大（v0对比v3）。
- 使用SimHash+Multi-Head的两阶段方式，比直接使用Avg pooling方式效果更好，而且时间开销也相差不大（v0对比v1）。
- 随着序列长度的增加，SimHash的效果也变得更好，但是耗时也略微上涨（v2.1~v2.3）

ETA使用SimHash检索，在效果没有明显下降的同时，耗时有明显下降（从35ms降低到19ms），AUC只降低了0.1%左右。

<img src="../data/img/eta_vs.png" style="zoom:50%"/>